<a href="https://colab.research.google.com/github/bomiiisong/AI_Project/blob/master/Scrap/instagram_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instagram Scraper

## 1. Import Packages

In [ ]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import requests
import shutil
from selenium.webdriver.common.keys import Keys
import time as time
import getpass

## 2. Scrap Code

In [ ]:
# Search with Tag
baseUrl = 'https://www.instagram.com/explore/tags/'
plusUrl = input('검색할 태그를 입력하세요 : ')
url = baseUrl + quote_plus(plusUrl)

# Search with User
baseUrl = 'https://www.instagram.com/'
Username = input('Username : ')
url = baseUrl + quote_plus(Username)

# ChromeDriver 실행
driver = webdriver.Chrome("chromedriver")
time.sleep(3)

# Instagram Login

driver.get('https://www.instagram.com/accounts/login/')

username = getpass.getpass("Input ID : ")# User ID
password = getpass.getpass("Input PWD : ")# User PWD, getpass 이용 시, 정보를 숨길 수 있어 유용
 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
time.sleep(3)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)

# Login 버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

time.sleep(3)

# url 이동
driver.get(url)

time.sleep(3)

# Image link 담을 list
imglist = []

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    img_selector = soup.findAll('div', {'class' : 'KL4Bh'})
    
    time.sleep(3)
    
    for i, v in enumerate(img_selector):
        img = v.find('img')
        img_source = img.get('src')
        imglist.append(img_source)

    time.sleep(3)

# 페이지 스크롤    
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        
        else:
            last_height = new_height
            continue

driver.close()

## 3. Save Image

In [ ]:
# image 저장 번호
n = 1
# 중복 image 없도록 중복제거
imglist = list(set(imglist))

# image list 길이만큼 반복하며 저장
for i in range(len(imglist)):

    # This is the image url.
    image_url = imglist[i]
    resp = requests.get(image_url, stream = True)

    # image 저장 -> img 폴더 아래 저장 (.ipynb 파일과 동일한 경로에 img 폴더 미리 생성해놓기)
    local_file = open('./img/' + plusUrl+ str(n) + '.jpg', 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    
    n += 1
    del resp